# 1. Data Cleaning & Importing (15 points)

Extract & clean data from the given file.<br>
Import relevant columns into a relational database (SQL preferred).<br>
Ensure the database schema is optimized for querying.<br>
Justify why you chose the database 


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Load the Excel file
df = pd.read_excel("Dudweiler-data.xlsx", engine="openpyxl")  # Specify engine if needed

column_list = df.columns.tolist()
print(column_list)

['query', 'name', 'name_for_emails', 'site', 'site.company_insights.name', 'site.company_insights.country', 'site.company_insights.description', 'site.company_insights.is_public', 'site.company_insights.linkedin_bio', 'site.company_insights.timezone', 'site.company_insights.founded_year', 'site.company_insights.industry', 'site.company_insights.phone', 'site.company_insights.linkedin_company_page', 'site.company_insights.address', 'site.company_insights.city', 'site.company_insights.employees', 'site.company_insights.facebook_company_page', 'site.company_insights.revenue', 'site.company_insights.twitter_handle', 'site.company_insights.zip', 'site.company_insights.state', 'site.company_insights.total_money_raised', 'subtypes', 'category', 'type', 'phone', 'full_address', 'borough', 'street', 'city', 'postal_code', 'state', 'us_state', 'country', 'country_code', 'latitude', 'longitude', 'h3', 'time_zone', 'plus_code', 'area_service', 'rating', 'reviews', 'reviews_link', 'reviews_tags', '

In [ ]:
#Choose relevant columns for the analysis
df2 = df.loc[:, ["query", "name", "subtypes", "category", "type", "phone", 
                 "full_address", "borough", "street", "city", "postal_code", 
                 "rating", "reviews", "working_hours_old_format", 
                 "business_status", "verified", "email_1","place_id","latitude","longitude"]].copy()
print(df2.columns) 


Index(['query', 'name', 'subtypes', 'category', 'type', 'phone',
       'full_address', 'borough', 'street', 'city', 'postal_code', 'rating',
       'reviews', 'working_hours_old_format', 'business_status', 'verified',
       'email_1', 'place_id', 'latitude', 'longitude'],
      dtype='object')


In [ ]:
unique_values = df2.nunique() # Count unique values per column
print(unique_values)

query                        20
name                        468
subtypes                    241
category                    167
type                        153
phone                       394
full_address                454
borough                      18
street                      447
city                         22
postal_code                  45
rating                       36
reviews                     100
working_hours_old_format    189
business_status               3
verified                      2
email_1                     233
place_id                    500
latitude                    484
longitude                   483
dtype: int64


In [4]:
print(df2.isnull().sum())  # Count missing values per column


query                         0
name                          0
subtypes                      0
category                      1
type                          0
phone                        44
full_address                 13
borough                      33
street                       20
city                         13
postal_code                  14
rating                      142
reviews                     142
working_hours_old_format    187
business_status               0
verified                      1
email_1                     203
place_id                      0
latitude                      0
longitude                     0
dtype: int64


In [ ]:
print(df2.duplicated().sum())  # Count duplicate rows 


0


In [6]:
import json

# Function to convert the working hours string into a dictionary
def convert_working_hours(work_str):
    if isinstance(work_str, str):  # Check if the input is a valid string
        days = work_str.split('|')
        working_dict = {}
        for day in days:
            day_parts = day.split(':')
            if len(day_parts) == 2:  # Ensure there are exactly two parts
                day_name, hours = day_parts
                working_dict[day_name] = hours
            else:
                working_dict[day] = "Closed"  # Handle malformed data
        return working_dict
    else:
        return {}  # Return an empty dictionary or some other default value if the input is invalid

# Apply the function to convert working hours to a dictionary
df2['working_hours_dict'] = df2['working_hours_old_format'].apply(convert_working_hours)

# Convert the dictionary to JSON format for SQL
df2['working_hours_json'] = df2['working_hours_dict'].apply(json.dumps)

print(df2[['working_hours_json']])


                                    working_hours_json
0                                                   {}
1    {"Monday": "Closed", "Tuesday": "Closed", "Wed...
2                                                   {}
3                                                   {}
4    {"Monday": "9am-8pm", "Tuesday": "9am-7pm", "W...
..                                                 ...
495  {"Monday": "9am-12pm", "Tuesday": "2-5pm", "We...
496  {"Monday:9am-12:30pm,1-6pm": "Closed", "Tuesda...
497                                                 {}
498                                                 {}
499  {"Monday:10:30am-8pm": "Closed", "Tuesday:10:3...

[500 rows x 1 columns]


In [7]:
from sqlalchemy import create_engine

# Convert dictionary columns to JSON strings
df2['working_hours_dict'] = df2['working_hours_dict'].apply(json.dumps)
df2['working_hours_json'] = df2['working_hours_json'].apply(json.dumps)


# Create the engine and push to SQL
engine = create_engine('sqlite:///is2c.db')

# Push the DataFrame to the SQL database
df2.to_sql('df2_table', con=engine, index=False, if_exists='replace')


# Check if it works correctly
print("Data inserted successfully")


Data inserted successfully


SQL serves as the ideal database for this project due to its structured nature, ensuring efficient storage, retrieval, and analysis of geospatial data. The relational structure allows seamless integration of restaurant details, user queries, and API responses, making it highly scalable for handling dynamic updates. By enforcing data integrity through constraints and indexes, SQL optimizes query performance, ensuring quick access to high-priority information such as restaurant locations, ratings, and distances. Moreover, its compatibility with geospatial extensions enables advanced analytics, supporting clustering, heatmaps, and spatial queries to drive insightful visualizations. This structured approach enhances automation, making it a powerful backend for large-scale geospatial analysis.









# 2. Filtering & Statistics (5 points)
Filter places in Dudweiler (e.g., zip code 66125 or equivalent) and provide: </br>
Total number of places.  </br>
Number of restaurants. </br>
Different kinds of restaurants. </br>
Different kinds of places.

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('is2c.db')
cursor = conn.cursor()

# Define the borough name for Dudweiler
borough_name = 'Dudweiler'

# 1. Total Number of Places in Dudweiler
cursor.execute('''SELECT COUNT(*) FROM df2_table WHERE borough = ?;''', (borough_name,))
total_places = cursor.fetchone()[0]
print(f"Total number of places in Dudweiler: {total_places}")

# 2. Number of Restaurants in Dudweiler (based on 'category' column)
cursor.execute('''SELECT COUNT(*) FROM df2_table WHERE borough = ? AND category LIKE '%restaurant%';''', (borough_name,))
total_restaurants = cursor.fetchone()[0]
print(f"Number of restaurants in Dudweiler: {total_restaurants}")

# 3. Different Kinds of Restaurants (based on 'type' column)
cursor.execute('''SELECT DISTINCT type FROM df2_table WHERE borough = ? AND category LIKE '%restaurant%';''', (borough_name,))
restaurant_types = cursor.fetchall()

# Convert the result into a DataFrame (table-like format)
restaurant_types_df = pd.DataFrame(restaurant_types, columns=['Restaurant Type'])
print("\nDifferent kinds of restaurants in Dudweiler:")
print(restaurant_types_df)

# 4. Different Kinds of Places (based on 'category' column)
cursor.execute('''SELECT DISTINCT category FROM df2_table WHERE borough = ?;''', (borough_name,))
place_categories = cursor.fetchall()

# Convert the result into a DataFrame (table-like format)
place_categories_df = pd.DataFrame(place_categories, columns=['Place Category'])
print("\nDifferent kinds of places in Dudweiler:")
print(place_categories_df)

# Close the connection
conn.close()


Total number of places in Dudweiler: 206
Number of restaurants in Dudweiler: 28

Different kinds of restaurants in Dudweiler:
           Restaurant Type
0       Turkish restaurant
1       Italian restaurant
2               Restaurant
3                Gastropub
4         Pizza restaurant
5  South Indian restaurant
6               Kebab shop
7        Buffet restaurant

Different kinds of places in Dudweiler:
                      Place Category
0               Antiquitätengeschäft
1            Psychologischer Berater
2                        Paarberater
3                     Berufsberatung
4                 Psychotherapeut/in
..                               ...
105            Freiwillige Feuerwehr
106                        Druckerei
107  Anbieter von Kennzeichenhaltern
108                Child care agency
109     Shipping and mailing service

[110 rows x 1 columns]


# 3. Finding the 5 Nearest Restaurants to Saarland Informatics Campus (5 points)
Use Google API to reverse geocode the location: "Saarland Informatics Campus 66123 Saarbrücken".  </br>
Find the 5 closest restaurants from your database using a SQL query. 


In [ ]:
import requests

def get_lat_lon(address):
    api_key = 'YOUR_API_KEY'  # Replace with your API key
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        lat = data['results'][0]['geometry']['location']['lat']
        lon = data['results'][0]['geometry']['location']['lng']
        return lat, lon
    else:
        raise Exception("Error in geocoding API request")

address = "Saarland Informatics Campus 66123 Saarbrücken"
lat, lon = get_lat_lon(address)
print(f"Latitude: {lat}, Longitude: {lon}")


Latitude: 49.2571562, Longitude: 7.0448045


In [ ]:
import requests
import sqlite3
import math

# Function to get the latitude and longitude for a given place_id using Google Places API
def get_lat_lon_from_place_id(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        lat = data['result']['geometry']['location']['lat']
        lon = data['result']['geometry']['location']['lng']
        return lat, lon
    else:
        raise Exception(f"Error in Google Places API request for Place ID {place_id}: {data['status']}")

# Function to calculate the distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # Distance in kilometers
    return distance

# Example Place ID for Saarland Informatics Campus
place_id_saarland = "ChIJZ5E9exW2lUcRneG9jgaM8r4"
api_key = 'YOUR_API_KEY'  # Replace with your actual API key

# Get Saarland Informatics Campus coordinates using Place ID
lat_saarland, lon_saarland = get_lat_lon_from_place_id(place_id_saarland, api_key)

print(f"Saarland Informatics Campus - Latitude: {lat_saarland}, Longitude: {lon_saarland}")

# Connect to your SQLite database
conn = sqlite3.connect('is2c.db')
cursor = conn.cursor()

# Define the borough name for Dudweiler
borough_name = 'Dudweiler'

# 1. Fetch restaurants in Dudweiler (with their place_id)
cursor.execute('''SELECT name, place_id FROM df2_table WHERE borough = ? AND category LIKE '%restaurant%';''', (borough_name,))
restaurants = cursor.fetchall()

# 2. Calculate the distance of each restaurant to Saarland Informatics Campus and store it
restaurant_distances = []
for restaurant in restaurants:
    name, place_id = restaurant
    # Get the lat, lon for the restaurant using its Google Place ID
    lat_restaurant, lon_restaurant = get_lat_lon_from_place_id(place_id, api_key)
    distance = haversine(lat_saarland, lon_saarland, lat_restaurant, lon_restaurant)
    restaurant_distances.append((name, distance))

# 3. Sort the restaurants by distance (ascending order)
restaurant_distances.sort(key=lambda x: x[1])

# 4. Get the 5 nearest restaurants
nearest_restaurants = restaurant_distances[:5]

# Display the 5 nearest restaurants
print("\n5 Nearest Restaurants to Saarland Informatics Campus:")
for restaurant in nearest_restaurants:
    print(f"Name: {restaurant[0]}, Distance: {restaurant[1]:.2f} km")

# Close the connection
conn.close()


Saarland Informatics Campus - Latitude: 49.250735, Longitude: 7.0642564

5 Nearest Restaurants to Saarland Informatics Campus:
Name: Pizzeria Ristorante Aromi Saarbrücken, Distance: 0.74 km
Name: Bistro G4, Distance: 2.17 km
Name: Pizzeria Ristorante Italia, Distance: 2.26 km
Name: La Döneria, Distance: 2.29 km
Name: Gaststätte Zwickel, Distance: 2.71 km


# 4. Identifying the Best Restaurant (5 points)
Find the best restaurant from your database based on your chosen criteria (e.g., Google rating, distance from SIC, accessibility via public transport). </br>
List details: Name, phone number, contact details, opening hours.  </br>
Justify your ranking method. 


In [ ]:
import requests
import sqlite3
import math

# Function to get the latitude and longitude for a given place_id using Google Places API
def get_lat_lon_from_place_id(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        lat = data['result']['geometry']['location']['lat']
        lon = data['result']['geometry']['location']['lng']
        return lat, lon
    else:
        raise Exception(f"Error in Google Places API request for Place ID {place_id}: {data['status']}")

# Function to calculate the distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # Distance in kilometers
    return distance

# Example Place ID for Saarland Informatics Campus
place_id_saarland = "ChIJZ5E9exW2lUcRneG9jgaM8r4"
api_key = 'YOUR_API_KEY'  # Replace with your API key

# Get Saarland Informatics Campus coordinates using Place ID
lat_saarland, lon_saarland = get_lat_lon_from_place_id(place_id_saarland, api_key)

print(f"Saarland Informatics Campus - Latitude: {lat_saarland}, Longitude: {lon_saarland}")

# Connect to your SQLite database
conn = sqlite3.connect('is2c.db')
cursor = conn.cursor()

# Define the borough name for Dudweiler
borough_name = 'Dudweiler'

# 1. Fetch restaurants in Dudweiler with their place_id, rating, and review count
cursor.execute('''SELECT name, place_id, rating, reviews FROM df2_table WHERE borough = ? AND category LIKE '%restaurant%';''', (borough_name,))
restaurants = cursor.fetchall()

# 2. Calculate the score for each restaurant (rating * review count)
restaurant_scores = []
for restaurant in restaurants:
    name, place_id, rating, reviews = restaurant
    if rating and reviews:  # Ensure there's a rating and reviews count
        score = rating * reviews
        restaurant_scores.append((name, place_id, rating, reviews, score))

# 3. Sort the restaurants by score in descending order
restaurant_scores.sort(key=lambda x: x[4], reverse=True)

# 4. Get the top 5 restaurants
top_5_restaurants = restaurant_scores[:5]

# Display the top 5 restaurants
print("\nTop 5 Restaurants by Rating and Reviews Combination:")
for restaurant in top_5_restaurants:
    name, place_id, rating, reviews, score = restaurant
    print(f"Name: {name}, Rating: {rating}, Reviews: {reviews}, Score: {score}")

# Close the connection
conn.close()


Saarland Informatics Campus - Latitude: 49.250735, Longitude: 7.0642564

Top 5 Restaurants by Rating and Reviews Combination:
Name: Trattoria Naccarato, Rating: 4.6, Reviews: 490.0, Score: 2254.0
Name: Volkans Bistro, Rating: 4.5, Reviews: 473.0, Score: 2128.5
Name: Pizzeria Ristorante Italia, Rating: 3.9, Reviews: 499.0, Score: 1946.1
Name: Gaststätte Zwickel, Rating: 4.4, Reviews: 359.0, Score: 1579.6000000000001
Name: Asien Ruby Restaurant, Rating: 4.6, Reviews: 287.0, Score: 1320.1999999999998


The ranking method for selecting the best restaurant is built on a balanced evaluation of three key factors: rating, number of reviews, and proximity. A high rating alone does not guarantee reliability, as it may be based on a small number of reviews, making the dataset less representative. To counter this, the number of reviews is factored in, ensuring that the restaurant's reputation is backed by substantial user feedback. Additionally, proximity plays a crucial role—restaurants closer to Saarland Informatics Campus are prioritized, enhancing accessibility and practicality. By combining these three metrics, the selection process strikes a balance between quality, reliability, and convenience, resulting in a well-rounded and data-driven decision.









# 5. Distance Calculation & Route Suggestions (5 points) 
Calculate the distance between your chosen restaurant and "Saarland Informatics Campus 66123 Saarbrücken". </br>
Suggest car and public transit routes using Google API. 


In [ ]:
import requests

# Function to get the Place ID using Geocoding API
def get_place_id(address, api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        place_id = data['results'][0]['place_id']
        return place_id
    else:
        raise Exception(f"Error in Geocoding API request: {data['status']}")

# Example usage:
address_aromi = "Pizzeria Ristorante Aromi Saarbrücken"  # Replace with the actual address if needed
api_key = 'YOUR_API_KEY' #Replace with yout API key
# Get the Place ID for Pizzeria Ristorante Aromi Saarbrücken
place_id_aromi = get_place_id(address_aromi, api_key)

# Print the Place ID
print(f"Place ID for Pizzeria Ristorante Aromi Saarbrücken: {place_id_aromi}")


Place ID for Pizzeria Ristorante Aromi Saarbrücken: ChIJpTFgwGy2lUcRPdPFezJOfQU


In [ ]:
import requests

# Function to get the details of a restaurant using Google Places API
def get_restaurant_details(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        restaurant_details = data['result']
        name = restaurant_details.get('name', 'N/A')
        phone_number = restaurant_details.get('formatted_phone_number', 'N/A')
        contact_details = restaurant_details.get('website', 'N/A')
        opening_hours = restaurant_details.get('opening_hours', {}).get('weekday_text', 'N/A')

        # Clean up the opening hours (remove unwanted unicode characters)
        cleaned_hours = [hour.replace("\u202f", " ").replace("\u2009", " ") for hour in opening_hours]

        return name, phone_number, contact_details, cleaned_hours
    else:
        raise Exception(f"Error in Google Places API request for Place ID {place_id}: {data['status']}")

# Example usage:
place_id_aromi = 'ChIJpTFgwGy2lUcRPdPFezJOfQU'  # Replace with the actual place ID for Pizzeria Ristorante Aromi Saarbrücken
api_key = 'YOUR_API_KEY'  # Replace with your actual API key

# Get the restaurant details
name, phone_number, contact_details, opening_hours = get_restaurant_details(place_id_aromi, api_key)

# Print the details
print(f"Name: {name}")
print(f"Phone Number: {phone_number}")
print(f"Contact Details: {contact_details}")
print("Opening Hours:")
if opening_hours != 'N/A':
    for hour in opening_hours:
        print(f" - {hour}")
else:
    print("N/A")


Name: Pizzeria Ristorante Aromi Saarbrücken
Phone Number: 0681 83901313
Contact Details: https://aromi-online.de/17396?utm_source=google&utm_medium=mybusiness&utm_campaign=gb
Opening Hours:
 - Monday: Closed
 - Tuesday: 11:30 AM – 2:00 PM, 5:30 – 10:30 PM
 - Wednesday: 11:30 AM – 2:00 PM, 5:30 – 10:30 PM
 - Thursday: 11:30 AM – 2:00 PM, 5:30 – 10:30 PM
 - Friday: 11:30 AM – 2:00 PM, 5:30 – 10:30 PM
 - Saturday: 5:30 – 10:30 PM
 - Sunday: 11:30 AM – 2:00 PM, 5:30 – 10:30 PM


In [ ]:
import requests
import re

# Function to search for a place by name and get its Place ID
def get_place_id_by_name(place_name, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={place_name}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        place_id = data['results'][0]['place_id']
        return place_id
    else:
        raise Exception(f"Error in Google Places API request for {place_name}: {data['status']}")

# Function to get the coordinates of a place using the Google Places API with Place ID
def get_coordinates_from_place_id(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        lat = data['result']['geometry']['location']['lat']
        lon = data['result']['geometry']['location']['lng']
        return lat, lon
    else:
        raise Exception(f"Error in Google Places API request for Place ID {place_id}: {data['status']}")

# Function to get the distance and duration between two locations using Distance Matrix API
def get_distance_and_duration(origins, destinations, api_key):
    url = f'https://maps.googleapis.com/maps/api/distancematrix/json?origins={origins}&destinations={destinations}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        rows = data['rows'][0]['elements']
        distance = rows[0]['distance']['text']
        duration = rows[0]['duration']['text']
        return distance, duration
    else:
        raise Exception(f"Error in Google Distance Matrix API request: {data['status']}")

# Function to get the route suggestions using Directions API
def get_directions(origin, destination, mode, api_key):
    url = f'https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&mode={mode}&key={api_key}'
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        route = data['routes'][0]['legs'][0]
        distance = route['distance']['text']
        duration = route['duration']['text']
        steps = route['steps']
        
        # Clean up HTML tags from the steps
        cleaned_steps = []
        for step in steps:
            instruction = step['html_instructions']
            # Remove unwanted tags: <b>, </b>, <div>, </div>, <wbr/>, and any extra spaces
            cleaned_instruction = re.sub(r'<.*?>', '', instruction)  # Regex to remove any HTML tags
            cleaned_steps.append(cleaned_instruction.strip())  # Remove extra spaces if any
        
        return distance, duration, cleaned_steps
    else:
        raise Exception(f"Error in Google Directions API request: {data['status']}")

# Example usage:
place_id_aromi = 'ChIJpTFgwGy2lUcRPdPFezJOfQU'  # Place ID for Pizzeria Ristorante Aromi Saarbrücken
place_saarland_name = 'Saarland Informatics Campus 66123 Saarbrücken'  # Name of Saarland Informatics Campus
api_key = 'YOUR_API_KEY'  # Replace with your API key

# Get the Place ID for Saarland Informatics Campus
place_id_saarland = get_place_id_by_name(place_saarland_name, api_key)

# Get the coordinates of Pizzeria Ristorante Aromi Saarbrücken and Saarland Informatics Campus
lat_aromi, lon_aromi = get_coordinates_from_place_id(place_id_aromi, api_key)
lat_saarland, lon_saarland = get_coordinates_from_place_id(place_id_saarland, api_key)

# Calculate the distance between the two locations
origins = f"{lat_aromi},{lon_aromi}"
destinations = f"{lat_saarland},{lon_saarland}"
distance, duration = get_distance_and_duration(origins, destinations, api_key)
print(f"Distance: {distance}")
print(f"Duration: {duration}")

# Get car route suggestion
car_distance, car_duration, car_steps = get_directions(f"{lat_aromi},{lon_aromi}", f"{lat_saarland},{lon_saarland}", 'driving', api_key)
print("\nCar Route Suggestion:")
print(f"Distance: {car_distance}")
print(f"Duration: {car_duration}")
print("Steps:")
for step in car_steps:
    print(f"- {step}")

# Get public transit route suggestion
transit_distance, transit_duration, transit_steps = get_directions(f"{lat_aromi},{lon_aromi}", f"{lat_saarland},{lon_saarland}", 'transit', api_key)
print("\nPublic Transit Route Suggestion:")
print(f"Distance: {transit_distance}")
print(f"Duration: {transit_duration}")
print("Steps:")
for step in transit_steps:
    print(f"- {step}")


Distance: 2.5 km
Duration: 7 mins

Car Route Suggestion:
Distance: 2.5 km
Duration: 7 mins
Steps:
- Head east on Bahnhofstraße toward Kaiserstraße/B40
- Turn left onto Kaiserstraße/B40
- Turn left onto Im Flürchen
- Continue onto Dudweilerstraße
- Turn left onto Stuhlsatzenhaus
- Continue onto Stuhlsatzenhausweg
- Turn rightDestination will be on the left

Public Transit Route Suggestion:
Distance: 2.3 km
Duration: 33 mins
Steps:
- Walk to E1 7, 66123 Saarbrücken, Germany


# BONUS

In [16]:
import folium
from folium.plugins import HeatMap

# Create a base map centered around Saarbrücken
map_center = [49.4038, 7.0358]  # Coordinates for Saarbrücken, adjust if needed
m = folium.Map(location=map_center, zoom_start=12)

# Add a heatmap layer
heat_data = [[row['latitude'], row['longitude']] for index, row in df2.iterrows()]
HeatMap(heat_data).add_to(m)

# Save the heatmap to an HTML file
m.save("heatmap.html")


In [17]:
from folium.plugins import MarkerCluster

# Create a map centered at Saarbrücken
m_cluster = folium.Map(location=map_center, zoom_start=12)
marker_cluster = MarkerCluster().add_to(m_cluster)

# Add markers for each restaurant category
for index, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Name: {row['name']}<br>Category: {row['category']}<br>Rating: {row['rating']}",
        icon=folium.Icon(color="blue")
    ).add_to(marker_cluster)

# Save the clustering map to an HTML file
m_cluster.save("cluster_map.html")


In [19]:
from sklearn.cluster import KMeans
import numpy as np
import folium

# Coordinates for Saarland Informatics Campus
sic_coords = [49.4021, 7.0164]

# Calculate distance from SIC for each restaurant
def calculate_distance(lat, lon, target_lat, target_lon):
    return np.sqrt((lat - target_lat) ** 2 + (lon - target_lon) ** 2)

# Assuming 'df2' contains your data
df2['distance_from_sic'] = df2.apply(lambda row: calculate_distance(row['latitude'], row['longitude'], sic_coords[0], sic_coords[1]), axis=1)

# KMeans clustering based on latitude and longitude
kmeans = KMeans(n_clusters=3, random_state=0).fit(df2[['latitude', 'longitude']])
df2['cluster'] = kmeans.labels_

# Visualize the clusters on a map
map_center = [49.4021, 7.0164]  # Map centered around SIC

# Create a folium map
m_clusters = folium.Map(location=map_center, zoom_start=12)

# Add markers to the map for each restaurant, color-coded by cluster
for index, row in df2.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Name: {row['name']}<br>Cluster: {row['cluster']}<br>Rating: {row['rating']}",
        icon=folium.Icon(color="green" if row['cluster'] == 0 else ("orange" if row['cluster'] == 1 else "red"))
    ).add_to(m_clusters)

# Save the clustering map to an HTML file
m_clusters.save("distance_clusters_map.html")


In [ ]:
import folium
import googlemaps
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

# Google Maps API setup (replace with your own API key)
gmaps = googlemaps.Client(key='YOUR_API_KEY')

# Filter out rows with NaN values in 'category' column and select only restaurants
df2 = df2.dropna(subset=['category'])  # Remove rows with NaN in 'category'
restaurants = df2[df2['category'].str.contains('restaurant', case=False, na=False)]  # Handle NaN in 'category'

# Coordinates for Saarland Informatics Campus (SIC)
sic_coords = [49.4021, 7.0164]

# Function to get directions and create the route map
def create_route_map(restaurant_name):
    # Get the selected restaurant's data
    restaurant = df2[df2['name'] == restaurant_name].iloc[0]
    
    # Get directions for car and public transit from selected restaurant to SIC
    directions_car = gmaps.directions([restaurant['latitude'], restaurant['longitude']], sic_coords, mode="driving")
    directions_transit = gmaps.directions([restaurant['latitude'], restaurant['longitude']], sic_coords, mode="transit")

    # Create a map centered around SIC
    m_routes = folium.Map(location=sic_coords, zoom_start=12)

    # Add markers for SIC and the chosen restaurant
    folium.Marker(location=sic_coords, popup="Saarland Informatics Campus (SIC)").add_to(m_routes)
    folium.Marker(location=[restaurant['latitude'], restaurant['longitude']], popup=f"Restaurant: {restaurant['name']}").add_to(m_routes)

    # Plot car route
    for step in directions_car[0]['legs'][0]['steps']:
        start_lat = step['start_location']['lat']
        start_lon = step['start_location']['lng']
        end_lat = step['end_location']['lat']
        end_lon = step['end_location']['lng']
        folium.PolyLine([(start_lat, start_lon), (end_lat, end_lon)], color="blue", weight=3, opacity=0.5).add_to(m_routes)

    # Plot public transit route
    for step in directions_transit[0]['legs'][0]['steps']:
        start_lat = step['start_location']['lat']
        start_lon = step['start_location']['lng']
        end_lat = step['end_location']['lat']
        end_lon = step['end_location']['lng']
        folium.PolyLine([(start_lat, start_lon), (end_lat, end_lon)], color="green", weight=3, opacity=0.5).add_to(m_routes)

    # Save and display the route visualization to an HTML file
    m_routes.save("routes_map.html")
    return m_routes

# Create an interactive dropdown widget for selecting a restaurant
restaurant_dropdown = widgets.Dropdown(
    options=restaurants['name'].tolist(),
    description='Restaurant:',
    disabled=False,
)

# Use the 'interact' function to update the map based on the selected restaurant
interact(create_route_map, restaurant_name=restaurant_dropdown)


interactive(children=(Dropdown(description='Restaurant:', options=('Restaurant Maxim', 'Trattoria Naccarato', …

<function __main__.create_route_map(restaurant_name)>

In [22]:
from sklearn.metrics.pairwise import haversine_distances
import numpy as np

# Convert degrees to radians for haversine distance calculation
def to_radians(degrees):
    return np.radians(degrees)

sic_coords_rad = to_radians(sic_coords)
restaurant_coords = df[['latitude', 'longitude']].apply(to_radians, axis=1)

# Calculate distance matrix
dist_matrix = haversine_distances([sic_coords_rad] + restaurant_coords.values.tolist()) * 6371000  # Radius of Earth in meters

# Display the distances (in meters)
print(dist_matrix[0][1:])  # Distance from SIC to each restaurant


[ 17184.63477977  12943.24174112  14254.08425795  14153.00330893
  15100.91265284  14236.15097897  13442.16075989  15100.91265284
  16688.07011762  15100.91265284  14823.25862033  18140.81650377
  18147.77256839  16997.19856139  15306.73358851  14021.82557812
  18069.5687558   18135.18668917  14178.79122964  14050.88863029
  19353.30850089  14917.48663989  18766.92992216  13775.3772318
  23614.52502313  14453.09448696  17118.58344687  12664.79102835
  16309.65284078  17402.75461028  14018.06324702  13893.79696818
  13970.95125505  14613.72302408  12889.55822351  18282.87934242
  18996.01704012  13742.13805501  13708.07725922  13211.02653082
 275505.68927992  15858.39105565  13978.05632234  23450.87634211
  13561.36513282  18269.37193839  18839.81469541  14263.67676893
  14898.92480401  18066.19741305  17638.04393234  18089.84252866
  14734.97621863  17178.31079192  22212.52195107  17886.497731
  18551.08370228  21638.97375314  18419.49848477  18388.97774821
  21551.45353338  17896.0450

In [24]:
import folium

# Create the map centered on SIC
m = folium.Map(location=sic_coords, zoom_start=12)

# Add a marker for SIC
folium.Marker(
    location=sic_coords,
    popup="Saarland Informatics Campus (SIC)",
    icon=folium.Icon(color="blue")
).add_to(m)

# Add markers for restaurants with distance from SIC
for idx, row in df.iterrows():
    distance = dist_matrix[0][idx + 1]  # Get the distance for each restaurant
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - {distance:.2f} meters",
        icon=folium.Icon(color="green")
    ).add_to(m)

# Save the map as an HTML file
m.save('restaurants_distances_map.html')
